In [1]:
#Data downloaded from https://janelia.figshare.com/articles/dataset/Recordings_of_ten_thousand_neurons_in_visual_cortex_in_response_to_2_800_natural_images/6845348

In [24]:
from EnsemblePursuit.EnsemblePursuit import EnsemblePursuit
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.linear_model import ridge_regression
import torch
from scipy.ndimage import gaussian_filter
import cv2

In [117]:
dat_path='C:/Users/koester_lab/Documents/Maria/CarsenData/2800/natimg2800_M170717_MP033_2017-08-20.mat'
dat_path='C:/Users/koester_lab/Documents/Maria/CarsenData/2800/natimg2800_M170717_MP034_2017-09-11.mat'
im_path='C:/Users/koester_lab/Downloads/images_natimg2800_all.mat'

In [118]:
from scipy.stats import zscore
from scipy.sparse.linalg import eigsh


def subtract_spont(spont,resp):
    #print(spont)
    mu = spont.mean(axis=0)
    sd = spont.std(axis=0) + 1e-6
    resp = (resp - mu) / sd
    spont = (spont - mu) / sd
    sv,u = eigsh(spont.T @ spont, k=32)
    resp = resp - (resp @ u) @ u.T
    return resp

def test_train_split(data,stim):
    unique, counts = np.unique(stim.flatten(), return_counts=True)
    count_dict=dict(zip(unique, counts))

    keys_with_enough_data=[]
    for key in count_dict.keys():
        if count_dict[key]==2:
            keys_with_enough_data.append(key)

    filtered_stims=np.isin(stim.flatten(),keys_with_enough_data)

    #Arrange data so that responses with the same stimulus are adjacent
    z=stim.flatten()[np.where(filtered_stims)[0]]
    sortd=np.argsort(z)
    istim=np.sort(z)
    X=data[filtered_stims,:]
    out=X[sortd,:].copy()

    x_train=out[::2,:]
    y_train=istim[::2]
    x_test=out[1::2,:]
    y_test=istim[1::2]

    return x_train, x_test, y_train, y_test

def zscore_utils(X, axis=0):
    mean_X= np.mean(X,axis=axis)
    std_X = np.std(X, axis=axis) + 1e-10
    X -= np.expand_dims(mean_X, axis)
    X /= np.expand_dims(std_X, axis)

    return X

def plot_all_receptive_fields(B):
    fig=plt.figure(figsize=(15, 10))
    for ind1 in range(50):
        vmax=np.max(B) * 2/3
        plt.subplot(10, 5, ind1+1)
        plt.imshow(B[:,:,ind1],cmap='bwr',vmin=-vmax,vmax=vmax)
        plt.axis('off')
        plt.title(str(ind1))

def fit_ridge_with_plot(V,im_path,dat_path):
    
    img=loadmat(im_path)['imgs']
    data=loadmat(dat_path)
    resp = data['stim'][0]['resp'][0]
    spont =data['stim'][0]['spont'][0]
    X=subtract_spont(spont,resp)
    X = zscore_utils(X, axis=1)
    model=EnsemblePursuit(n_components=50,lam=0.01,n_kmeans=50)
    model.fit(X)
    V = model.components_
    stim = data['stim']['istim'][0][0].astype(np.int32)
    print(stim.shape)
    ishape = (img.shape[0]//5, img.shape[1]//5)
    images = np.zeros((2800, ) + (ishape[0], ishape[1]))
    for j in range(images.shape[0]):
        images[j] = cv2.resize(img[:,:,j], (ishape[1], ishape[0]))
    images = images.reshape((2800, -1))
    images = zscore_utils(images, axis=1)
    #stim = data['stim']['istim'][0][0].astype(np.int32)
    x_train,x_test,y_train,y_test=test_train_split(V,stim)
    y_train=y_train-1
    images_ = images[y_train]
    print(images_.shape)
    av_resp = (x_train + x_test)/2
    B = np.linalg.solve(images_.T @ images_ +  100 * np.eye(images_.shape[1]), images_.T @ av_resp)
    print(B.shape)
    B = np.reshape(B, ishape+(50,))
    print(B.shape)
    B = gaussian_filter(B, (.5, .5, 0))
    plot_all_receptive_fields(B)

In [ ]:
fit_ridge_with_plot(V,im_path,dat_path)

obtained 50 PCs in 1.0423 seconds
initialized 50 clusters with k-means in 9.4485 seconds
ensemble 0, time 2.18, nr neurons 1452, EV 0.0042
